![](https://i.imgur.com/qkg2E2D.png)

# UnSupervised Learning Methods

## Exercise 004 - Part III

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 16/06/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/UnSupervisedLearningMethods/2023_03/Exercise0004Part003.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.cluster import KMeans
from sklearn.datasets import make_s_curve, make_swiss_roll
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import kneighbors_graph
from scipy.sparse.csgraph import connected_components, shortest_path

# Computer Vision

# Miscellaneous
import os
import math
from platform import python_version
import random
import time
import urllib.request

# Typing
from typing import Callable, List, Tuple, Union

# Visualizatioblankn
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

from CourseAuxFun_GRP_F import *


## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

DATA_FILE_URL   = r'https://drive.google.com/uc?export=download&confirm=9iBg&id=1lT6eWVRvfx_iusI9zB1fLg_g64n_141A'
DATA_FILE_NAME  = r'Rings5.mat'


In [ ]:
# Auxiliary Functions

def Plot3DScatter(mX: np.ndarray, hA: plt.Axes, vC: np.ndarray = None) -> None:
    m = mX.min()
    M = mX.max()
    if vC is not None:
        hA.scatter(*mX.T, s = 50,  c = vC, edgecolor = 'k', alpha = 1)
    else:
        hA.scatter(*mX.T, s = 50,  c = 'lime', edgecolor = 'k', alpha = 1)
    hA.set_xlim([m, M])
    hA.set_ylim([m, M])
    hA.set_zlim([m, M])
    hA.set_xlabel('$x_1$')
    hA.set_ylabel('$x_2$')
    hA.set_zlabel('$x_3$')

def Plot2DScatter(mX: np.ndarray, hA: plt.Axes, vC: np.ndarray = None) -> None:
    m = mX.min()
    M = mX.max()
    if vC is not None:
        hA.scatter(*mX.T, s = 50,  c = vC, edgecolor = 'k', alpha = 1)
    else:
        hA.scatter(*mX.T, s = 50,  c = 'lime', edgecolor = 'k', alpha = 1)
    hA.set_xlim([m, M])
    hA.set_ylim([m, M])
    hA.set_xlabel('$x_1$')
    hA.set_ylabel('$x_2$')


def MaximumSparseMat(mA: sp.sparse.spmatrix, mB: sp.sparse.spmatrix) -> sp.sparse.spmatrix:
    """
    Returns the element wise maximum of sparse matrices `mA` and `mB`.
    """
    mAgtB = (mA > mB).astype(int)
    mM = mAgtB.multiply(mA - mB) + mB

    return mM


## Guidelines

 - Fill the full names and ID's of the team members in the `Team Members` section.
 - Answer all questions / tasks within the Jupyter Notebook.
 - Use MarkDown + MathJaX + Code to answer.
 - Verify the rendering on VS Code.
 - Submission in groups (Single submission per group).
 - You may and _should_ use the forums for questions.
 - Good Luck!

* <font color='brown'>(**#**)</font> The `Import Packages` section above imports most needed tools to apply the work. Please use it.
* <font color='brown'>(**#**)</font> You may replace the suggested functions to use with functions from other packages.
* <font color='brown'>(**#**)</font> Whatever not said explicitly to implement maybe used by a 3rd party packages.
* <font color='brown'>(**#**)</font> The total run time of this notebook must be **lower than 60 [Sec]**.

## Team Members

- Nadav_Talmon_203663950
- Nadav_Shaked_312494925
- Adi_Rosenthal_316550797

## Generate / Load Data

In [ ]:
# Download Data
# This section downloads data from the given URL if needed.

if (DATA_FILE_NAME != 'None') and (not os.path.exists(DATA_FILE_NAME)):
    urllib.request.urlretrieve(DATA_FILE_URL, DATA_FILE_NAME)

## 8. IsoMap & Laplacian EigenMaps

### 8.1. IsoMap Algorithm (Bonus 4 Points)

In this section we'll implement a SciKit Learn API compatible class for the IsoMap algorithm.  
For the graphs we'll use the _K Nearest Neighbors_ approach.

The class should implement the following methods:

1. `__init____()` - The object constructor by the encoder dimension.  
2. `fit()` - Given a data set builds the encoder.  
3. `transform()` - Applies the encoding on the input data in out of sample manner.  
4. `fix_transform()` - Given a data set builds the encoder and applies the encoding.  

* <font color='brown'>(**#**)</font> Pay attention to data structure (`Nx x Nx` / `Nx x Ny`).
* <font color='brown'>(**#**)</font> Do not use any loops in you implementation.
* <font color='brown'>(**#**)</font> You should use your `CMDS()` implementation.
* <font color='brown'>(**#**)</font> Use `from CourseAuxFun.py import *` to import your code.
* <font color='brown'>(**#**)</font> You should use SciKit Learn's `sklearn.neighbors.kneighbors_graph`. Pay attention the output is a sparse matrix.
* <font color='brown'>(**#**)</font> You should use SciPy's `scipy.sparse.csgraph.connected_components` to check the graph is valid (Connected).  
* <font color='brown'>(**#**)</font> You should use SciPy's `scipy.sparse.csgraph.shortest_path` to computer the pairs shortest path matrix.  
* <font color='brown'>(**#**)</font> For the `transform()` methods you should use SciKit Learn's `sklearn.neighbors.NearestNeighbors`.  

**Remark**: The bonus is for manual implementation. You may use SciKit Learn's [Isomap]() without the bonus.

In [ ]:
class IsoMap():
    def __init__(self, d: int = 2, k: int = 9):
        '''
        Constructing the object.
        Args:
            d - Number of dimensions of the encoder output.
            k - Number of neighbors in the graph construction.
        '''
        # ===========================Fill This===========================#
        # 1. Keep the model parameters.

        self.d = d
        self.k = k
        self.cmds = CMDS(d=self.d)
        self.mDxx = None
        self.mX = None
        self.NN_mX = None
        self.encoder = None

        # ===============================================================#

    def fit(self, mX: np.ndarray):
        '''
        Fitting model parameters to the input.
        Args:
            mX - Input data with shape N x D.
        Output:
            self
        '''
        # ===========================Fill This===========================#
        # 1. Build the graph from the data.
        # 2. Verify the graph is connected (Raise error if not).
        # 3. Build the encoder.
        # !! Use the K-NN method to build the graph.
        # !! Make sure the graph obeys the assumptions made.
        # !! The encoder should be based on the CMDS() class.

        self.mX = mX
        knn_graph = kneighbors_graph(self.mX, self.k, mode='distance', include_self=False)
        n_components = connected_components(csgraph=knn_graph, directed=False, return_labels=False)
        if n_components > 1:
            raise ValueError("Graph is not connected")

        matrix_shortest_path = shortest_path(csgraph=knn_graph, directed=False)
        self.mDxx = matrix_shortest_path

        self.encoder = self.cmds.fit(self.mDxx).encoder

        # ===============================================================#
        return self

    def transform(self, mY: np.ndarray) -> np.ndarray:
        '''
        Applies (Out of sample) encoding.
        Args:
            mY - Input data (Out of Sample) with shape N x D.
        Output:
            mZ - Low dimensional representation (embeddings) with shape N x d.
        '''
        # ===========================Fill This===========================#
        # 1. Encode data using the model encoder.

        if self.NN_mX is None:
            NN_mX = NearestNeighbors(n_neighbors=1)
            NN_mX.fit(self.mX)
            self.NN_mX = NN_mX

        NearestNeighbor, distances = self.NN_mX.kneighbors(mY, return_distance=True)
        Dxy = self.mDxx[NearestNeighbor[0]] + distances[0]
        mZ = self.cmds.transform(Dxy)

        # ===============================================================#

        return mZ

    def fit_transform(self, mX: np.ndarray) -> np.ndarray:
        '''
        Applies encoding on the input.
        Args:
            mX - Input data (Distance matrix) with shape N x D.
        Output:
            mZ - Low dimensional representation (embeddings) with shape N x d.
        '''
        # ===========================Fill This===========================#
        # 1. Apply the `fit()` method.
        # 2. Encode data using the model encoder.

        self.fit(mX)
        mZ = self.encoder

        # ===============================================================#

        return mZ

> Royi: <font color='green'>✓ +4 Points (Bonus)</font>.  

* <font color='red'>(**?**)</font> Will `fit()` and then `transform()` will match the result of `fit_transform()`?  
  Make sure you understand this before proceeding.

### 8.2. Laplacian EigenMaps Algorithm

In this section we'll implement a SciKit Learn API compatible class for the Laplacian EigenMaps.  

The class should implement the following methods:

1. `__init____()` - The object constructor by the encoder dimension.  
2. `fit()` - Given a data set ($\boldsymbol{D}_{xx}$) builds the encoder.    
4. `fit_transform()` - Given a data set ($\boldsymbol{D}_{xx}$) builds the encoder and applies the encoding.  

* <font color='brown'>(**#**)</font> Pay attention to data structure (`Nx x Nx` / `Nx x Ny`).
* <font color='brown'>(**#**)</font> Do not use any loops in you implementation beside the main MM loop.
* <font color='brown'>(**#**)</font> Think about the difference in `transform()` and `fit_transform()` compared to `CMDS()` above.

In [ ]:
class LaplacianEigenMaps():
    def __init__(self, d: int = 2, k: int = 9, σ: float = 1):
        '''
        Constructing the object.
        Args:
            d - Number of dimensions of the encoder output.
            k - Number of neighbors in the graph construction.
            σ - The factor to multiply the median distance by.
        '''
        # ===========================Fill This===========================#
        # 1. Keep the model parameters.

        self.d = d
        self.k = k
        self.σ = σ
        self.mX = None
        self.knn_graph = None

        # ===============================================================#

    def fit(self, mX: np.ndarray):
        '''
        Fitting model parameters to the input.
        Args:
            mX - Input data with shape N x D.
        Output:
            self
        '''
        # ===========================Fill This===========================#
        # 1. Build a valid graph.
        # 2. Calculate the number of connected components in the graph.
        # 3. Keep the parameters in the object.
        # !! Raise error if the graph has more than 1 connected component.

        self.mX = mX

        knn_graph = kneighbors_graph(self.mX, self.k, mode='distance', metric="euclidean", include_self=False)
        n_components = connected_components(csgraph=knn_graph, directed=False, return_labels=False)
        if n_components > 1:
            raise ValueError("Graph is not connected")

        self.knn_graph = knn_graph

        # ===============================================================#
        return self

    def fit_transform(self, mX: np.ndarray) -> np.ndarray:
        '''
        Applies encoding on input data.
        Args:
            mX - Input data (Distance matrix) with shape N x D.
        Output:
            mZ - Low dimensional representation (embeddings) with shape N x d.
        '''
        # ===========================Fill This===========================#
        # 1. Apply the `fit()` method.
        # 2. Build the distance matrix.
        # 3. Set σ^2 to be the median squared euclidean distance multiplied by `self.σ^2``.
        # 4. Build the Affinity Matrix using Gaussian Weights.
        # 5. Build the Laplacian.
        # 6. Apply eigen decomposition to the Laplacian.
        # 7. Choose the eigen vectors wisely.
        # 8. Encode data.
        # !! You should chose the vectors

        self.fit(mX)
        knn_graph_dist = self.knn_graph.toarray()
        knn_graph_dist_symetric = np.max((knn_graph_dist.reshape(1, -1), knn_graph_dist.T.reshape(1, -1)),
                                         axis=0).reshape(knn_graph_dist.shape)
        W = np.zeros(knn_graph_dist_symetric.shape)
        W[knn_graph_dist_symetric != 0] = np.e ** (
                    -1 * (knn_graph_dist_symetric[knn_graph_dist_symetric != 0] ** 2) / (2 * self.σ ** 2))
        D = np.diag(np.sum(W, axis=1))
        L = D - W
        eigenvalues, eigenvectors = np.linalg.eig(L)
        sorted_eigenvalues = np.argsort(eigenvalues, axis=0)
        mZ = eigenvectors[:, sorted_eigenvalues[1:self.d + 1]]

        # ===============================================================#

        return mZ

* <font color='red'>(**?**)</font> Why is the `transform()` method not asked to be implemented?  
  Make sure you understand this before proceeding.

### 8.3. Clustering Using Dimensionality Reduction

In this section the IsMap and Laplacian Eigenmaps methods will be used for clustering of the _5 Rings_ data set.

For each data set:

1. Plot the Data Set  
   Plot the Data set in 3D.  
   **This is implemented**.
2. Reduce the Dimensionality of the Data  
   Reduce the dimensionality of the data to `d = 2` using each method.  
   Set the number of neighbors in the graph so the graph has a single component.
3. Apply the Clustering  
   Use K-Means for clustering with `k = 5`.  
4. Plot the Clustered Data  
   Plot the data with the clustering per method.  
   Plot the transformed labeled data and the original data per method.

* <font color='brown'>(**#**)</font> Pay attention to the difference in dimensions of the data to the derived Math formulations.
* <font color='brown'>(**#**)</font> The output should be 2 figures for each data set. You may show them in a single plot using sub plots.

In [ ]:
# Generate Data

mX = sp.io.loadmat('Rings5.mat')['pointCloud']
mX = mX.T

print(f'The data dimensions are {mX.shape[0]}x{mX.shape[1]}')

In [ ]:
# Plot Data
# Plotting the Data and a reference clustering by K-Means

K = 5 #<! Number of clusters

oKMeansCluster  = KMeans(n_clusters = K, n_init = 'auto')
vC              = oKMeansCluster.fit_predict(mX)

hF  = plt.figure(figsize = (12, 6))
hA1 = hF.add_subplot(1, 2, 1, projection = '3d')
hA2 = hF.add_subplot(1, 2, 2, projection = '3d')

Plot3DScatter(mX, hA1)
hA1.set_title('The 5 Rings Data')
hA1.view_init(elev = 45, azim = 300)

Plot3DScatter(mX, hA2, vC = vC)
hA2.set_title(f'The 5 Rings Data - Clustered by K-Means with K = {K}')
hA2.view_init(elev = 45, azim = 300)

plt.tight_layout()
plt.show()

In [ ]:
#===========================Fill This===========================#
# 1. Set parameters: `d`, `kNumNeighbors`, `σ` (Try to get a good clustering result).
# 2. Apply Dimensionality Reduction using IsoMap and Laplacian Eigen Maps.
# 3. Apply K-Means on the transformed data.
# 4. Display the clustered data in 2D (Low Dimension) and 3D (Original).
# !! You should use, in this case, the same number of neighbors for both algorithms.
# !! The output should be a figure of 2x2 axes (2D Scatter and 3D Scatter per method).
# !! You may use `Plot3DScatter()` and `Plot2DScatter()` for displaying the the data.

d = 2
kNumNeighbors = 40
σ = 0.2

isoMap = IsoMap(d=d, k=kNumNeighbors)
laplacianEigenMaps = LaplacianEigenMaps(d=d, k=kNumNeighbors, σ=σ)

isoMap_Z = isoMap.fit_transform(mX)
laplacianEigenMaps_Z = laplacianEigenMaps.fit_transform(mX)

# Perform clustering using K-Means on the transformed data

kmeans_isoMap = KMeans(n_clusters=K, random_state=seedNum, n_init='auto')
kmeans_laplacianEigenMaps = KMeans(n_clusters=K, random_state=seedNum, n_init='auto')

isoMap_clusters = kmeans_isoMap.fit_predict(isoMap_Z)
laplacianEigenMaps_clusters = kmeans_laplacianEigenMaps.fit_predict(laplacianEigenMaps_Z)

# Plot the clustered data in 2D and 3D
hF = plt.figure(figsize=(12, 12))
hA1 = hF.add_subplot(2, 2, 1)
hA2 = hF.add_subplot(2, 2, 2, projection='3d')
hA3 = hF.add_subplot(2, 2, 3)
hA4 = hF.add_subplot(2, 2, 4, projection='3d')
# IsoMap
hA1.scatter(isoMap_Z[:, 0], isoMap_Z[:, 1], c=isoMap_clusters)
hA1.set_title('IsoMap - 2D Scatter')

Plot3DScatter(mX, hA2, vC=isoMap_clusters)
hA2.set_title('IsoMap - 3D Scatter')

# Laplacian EigenMaps
hA3.scatter(laplacianEigenMaps_Z[:, 0], laplacianEigenMaps_Z[:, 1], c=laplacianEigenMaps_clusters)
hA3.set_title('Laplacian EigenMaps - 2D Scatter')

Plot3DScatter(mX, hA4, vC=laplacianEigenMaps_clusters)
hA4.set_title('Laplacian EigenMaps - 3D Scatter')

plt.tight_layout()
plt.show()

#===============================================================#

### 8.4. Question

In the above we used _Laplacian Eigenmaps_ for dimensionality reduction and then clustering.  
What would change if the task was to apply Spectral Clustering?  
Describe what will happen for the data above (The _5 Rings_).  
Address the changes needed in the implementation of the class `LaplcaianEigenMaps()` and the use of the class.

* <font color='brown'>(**#**)</font> You should use the ideas in _Question 5.2._.

### 8.4. Solution
To apply Spectral Clustering using the LaplacianEigenmaps class, we don't need to make any changes to the class implementation. Simply create an instance of the class with d=num classes, fit the data, and then apply K-means clustering directly to the resulting high-dimensional data (in this case, 5-dimensional). The resulting cluster labels will provide the clustering assignments for the data.

---